In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
sys.path

['/Users/syeehyn/Desktop/fall2020/ieor4571/personalization/eggs',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python38.zip',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/lib-dynload',
 '',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages',
 '/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages/IPython/extensions',
 '/Users/syeehyn/.ipython',
 '../']

In [24]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from tqdm import tqdm

In [5]:
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [6]:
datas = loading(spark, '../data/interim')

In [7]:
datas

{'train_0.5_0.5': DataFrame[userId: string, movieId: string, rating: string],
 'train_0.25_0.75': DataFrame[userId: string, movieId: string, rating: string],
 'test_0.5_0.5': DataFrame[userId: string, movieId: string, rating: string],
 'test_0.25_0.75': DataFrame[userId: string, movieId: string, rating: string],
 'train_0.75_0.25': DataFrame[userId: string, movieId: string, rating: string],
 'test_0.75_0.25': DataFrame[userId: string, movieId: string, rating: string]}

In [8]:
cast_int = lambda df: df.select([F.col(c).cast('int') for c in df.columns])
training, test = cast_int(datas['train_0.75_0.25']), cast_int(datas['test_0.75_0.25'])

In [9]:
training = training.sample(fraction = .01)
test = test.sample(fraction = .01)

In [10]:
print(f'''
        training set num of rows {training.count()},
        test set num of rows {test.count()},
        training set num of users {training.select('userId').distinct().count()},
        training set num of movies {training.select('movieId').distinct().count()},
        test set num of users {test.select('userId').distinct().count()},
        test set num of movies {test.select('movieId').distinct().count()},
        ''')


        training set num of rows 923,
        test set num of rows 290,
        training set num of users 895,
        training set num of movies 586,
        test set num of users 290,
        test set num of movies 249,
        


In [11]:
maxIter = [2, 4, 6, 8, 10]
regParam = [1e-2, 1e-1, 1e0, 1e1, 1e2]
rank = [10, 50, 100, 150, 200]

In [17]:
param_list = [(i, j, k) for i in maxIter for j in regParam for k in rank]

In [26]:
ALS?

In [27]:
_training, _validation = training.randomSplit([.9, .1], seed = 0)
evaluator=RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmses = []
for i,j,k in tqdm(param_list):
    als = ALS(userCol="userId", 
              itemCol="movieId", 
              ratingCol="rating",
              coldStartStrategy="drop", 
              nonnegative=True,
              maxIter=i,
              regParam=j,
              rank=k)
    model = als.fit(_training)
    rmses.append(evaluator.evaluate(model.transform(_validation)))

  1%|          | 1/125 [00:02<04:50,  2.34s/it]Exception ignored in: <function tqdm.__del__ at 0x7fae70bff820>
Traceback (most recent call last):
  File "/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages/tqdm/std.py", line 1128, in __del__
    self.close()
  File "/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages/tqdm/notebook.py", line 261, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'
Exception ignored in: <function JavaWrapper.__del__ at 0x7fae70f13430>
Traceback (most recent call last):
  File "/Users/syeehyn/opt/anaconda3/envs/personalization/lib/python3.8/site-packages/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'ALS' object has no attribute '_java_obj'
100%|██████████| 125/125 [05:27<00:00,  2.62s/it]


In [31]:
res = dict(zip(param_list, rmses))

In [33]:
import pandas as pd

In [45]:
out = pd.Series(res, name='rmse').to_frame().reset_index()
out.columns = ['maxIter', 'regParam', 'rank', 'rmse']

In [46]:
out

,maxIter,regParam,rank,rmse
0,2,0.01,10,4.374109
1,2,0.01,50,4.587460
2,2,0.01,100,4.669366
3,2,0.01,150,4.756296
4,2,0.01,200,4.572058
...,...,...,...,...
120,10,100.00,10,5.000000
121,10,100.00,50,5.000000
122,10,100.00,100,5.000000
123,10,100.00,150,5.000000
